In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sacremoses
!pip install transformers
!pip install torch
!pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=041ed5411e6543febd3646acf7322865152fdf6e61ce57b0faa13c29a0c6b0e5
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.7 MB/s eta 0:00:00


In [ ]:
import logging
import warnings
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import torch
import pandas as pd
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import MarianMTModel, MarianTokenizer
import warnings
warnings.filterwarnings('ignore')


# Set the logging level for the transformers module
logging.getLogger("transformers").setLevel(logging.ERROR)
# Suppress specific FutureWarning messages
warnings.filterwarnings("ignore", category=FutureWarning, message="The `pad_to_max_length` argument is deprecated*")

# Initialize an empty list to store comments
comments = []


label_dict = {0: 0, 1: 1}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                          num_labels=len(label_dict),
                                                          output_attentions=False,
                                                          output_hidden_states=False)

model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',do_lower_case=True)
model.load_state_dict(torch.load('/content/drive/MyDrive/MyModel_BERT_epoch_4.model', map_location=torch.device('cuda')))

#t5-small model
fine_tuned_model_t5small = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/fine_tuned_t5_small").to(device)
fine_tuned_tokenizer_t5small = T5Tokenizer.from_pretrained("/content/drive/MyDrive/fine_tuned_t5_small")

#t5-base model
fine_tuned_model_t5base = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/fine_tuned_t5_base").to(device)
fine_tuned_tokenizer_t5base = T5Tokenizer.from_pretrained("/content/drive/MyDrive/fine_tuned_t5_base")

print(fine_tuned_model_t5base.device)

#t5-large model
fine_tuned_model_t5large = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/fine_tuned_t5_large").to(device)
fine_tuned_tokenizer_t5large = T5Tokenizer.from_pretrained("/content/drive/MyDrive/fine_tuned_t5_large")

# Load the fine-tuned BART_large model
fine_tuned_bart_large = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/fine_tuned_bart_large").to(device)
tokenizer_Bart_large = BartTokenizer.from_pretrained("facebook/bart-large")

# Load the fine-tuned BART_large model
fine_tuned_bart_base = BartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/fine_tuned_bart_base").to(device)
tokenizer_Bart_base = BartTokenizer.from_pretrained("facebook/bart-base")

#Load the fine_tuned Nllb model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
fine_tuned_nllb_model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/nllb-200-distilled-600M_up").to(device)
tokenizer_nllb = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")

# Load the fine-tuned MarianMT model
fine_tuned_marian_model = MarianMTModel.from_pretrained("/content/drive/MyDrive/marian_output").to(device)
tokenizer_Marian = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ROMANCE")

cuda:0


In [ ]:
def test_model(dataloader_test):

    model.eval()

    predictions = []

    for batch in dataloader_test:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
                }
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)

    predictions = np.concatenate(predictions, axis=0)
    return predictions


In [ ]:
def detection(df):

    encoded_test_val = tokenizer.batch_encode_plus(
     df.message.values,
     add_special_tokens=True,
     return_attention_mask=True,
     pad_to_max_length=True,
     max_length=512,
     return_tensors='pt'
    )

    input_ids_test = encoded_test_val['input_ids']
    attention_masks_test = encoded_test_val['attention_mask']
    labels_test = torch.tensor(df.label.values)

    dataset_test = TensorDataset(input_ids_test, attention_masks_test,labels_test)
    batch_size = 8
    dataloader_test = DataLoader(dataset_test,
                                    sampler=SequentialSampler(dataset_test),
                                    batch_size=batch_size)


    pred_test = test_model(dataloader_test)
    preds_flat_test = np.argmax(pred_test, axis=1).flatten()
    return preds_flat_test
#     #print(preds_flat_test)

In [ ]:
def t5_small(c):
    # T5 model
    test_sentence = c
    inputs = fine_tuned_tokenizer_t5small.encode_plus(
        f"convert uncivil to civil: {test_sentence}",
        return_tensors="pt"
    )
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Ensure tensors are on the same device as the model (cuda or cpu)
    fine_tuned_model_t5small.to(device)

    output_ids = fine_tuned_model_t5small.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=1024,  # Adjust the max_length as needed
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    decoded_output = fine_tuned_tokenizer_t5small.decode(output_ids[0], skip_special_tokens=True)
    return decoded_output

In [ ]:
def t5_large(c):
    # T5 model
    test_sentence = c
    inputs = fine_tuned_tokenizer_t5large.encode_plus(
        f"convert uncivil to civil: {test_sentence}",
        return_tensors="pt"
    )
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Ensure tensors are on the same device as the model (cuda or cpu)
    fine_tuned_model_t5large.to(device)

    output_ids = fine_tuned_model_t5large.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=1024,  # Adjust the max_length as needed
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    decoded_output = fine_tuned_tokenizer_t5large.decode(output_ids[0], skip_special_tokens=True)
    return decoded_output

In [ ]:
def t5_base(c):
    # T5 model
    test_sentence = c
    inputs = fine_tuned_tokenizer_t5base.encode_plus(
        f"convert uncivil to civil: {test_sentence}",
        return_tensors="pt"
    )
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Ensure tensors are on the same device as the model (cuda or cpu)
    fine_tuned_model_t5base.to(device)

    output_ids = fine_tuned_model_t5base.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=1024,  # Adjust the max_length as needed
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        early_stopping=True
    )

    decoded_output = fine_tuned_tokenizer_t5base.decode(output_ids[0], skip_special_tokens=True)
    return decoded_output

In [ ]:
def Bart_large(c):
    uncivil_sentence = c
    inputs = tokenizer_Bart_large(uncivil_sentence, return_tensors="pt", max_length=1024, truncation=True).to(device)
    generated_ids = fine_tuned_bart_large.generate(**inputs, max_new_tokens=1024)
    generated_civil_sentence = tokenizer_Bart_large.decode(generated_ids[0], skip_special_tokens=True)
    return generated_civil_sentence

In [ ]:
def Bart_base(c):
    uncivil_sentence = c
    inputs = tokenizer_Bart_base(uncivil_sentence, return_tensors="pt", max_length=1024, truncation=True).to(device)
    generated_ids = fine_tuned_bart_base.generate(**inputs, max_new_tokens=1024)
    generated_civil_sentence = tokenizer_Bart_base.decode(generated_ids[0], skip_special_tokens=True)
    return generated_civil_sentence

In [ ]:
while True:
  # Collect comment from the user or exit the loop
  comment = input("Enter a comment (or type 'exit' to stop): ")

  if comment.lower() == 'exit':
    break

  # Add the comment to the list
  comments.append(comment)

  # Create a pandas DataFrame from the collected comments
  data = {'message': comments, 'label': 0}
  df = pd.DataFrame(data)

  preds_flat_test = detection(df)

  if preds_flat_test[-1]==0:
      print("\033[92mThis comment is professionally sound!\033[0m")
      print("\n")
  else:
      print("\033[91mThis comment is professionally inappropriate!\033[0m")
      print("\n")

      civil_options=0

      decoded_output = t5_large(comment)
      # Add the comment to the list
      comments.append(decoded_output)
      # Create a pandas DataFrame from the collected comments
      data = {'message': comments, 'label': 0}
      df = pd.DataFrame(data)

      preds_flat_test = detection(df)

      if preds_flat_test[-1]==0:
        civil_options+=1
        # print("\033[92mGenerated civil alternative-t5largel:\033[0m", decoded_output)
        st = "Option "+str(civil_options)+": "+decoded_output
        print(st)
        # print("\n")
      else:
        # print("\033[91mGenerated civil alternative-t5large:\033[0m", "Couldn't generate")
        # print("\n")

        decoded_output = t5_base(comment)
        # Add the comment to the list
        comments.append(decoded_output)
        # Create a pandas DataFrame from the collected comments
        data = {'message': comments, 'label': 0}
        df = pd.DataFrame(data)
        preds_flat_test = detection(df)

        if preds_flat_test[-1]==0:
          civil_options+=1
          # print("\033[92mGenerated civil alternative-t5base:\033[0m", decoded_output)
          st = "Option "+str(civil_options)+": "+decoded_output
          print(st)
          # print("\n")
        else:
          # print("\033[91mGenerated civil alternative-t5base:\033[0m", "Couldn't generate")
          # print("\n")
          decoded_output = t5_small(comment)
          # Add the comment to the list
          comments.append(decoded_output)
          # Create a pandas DataFrame from the collected comments
          data = {'message': comments, 'label': 0}
          df = pd.DataFrame(data)
          preds_flat_test = detection(df)
          if preds_flat_test[-1]==0:
            civil_options+=1
            st = "Option "+str(civil_options)+": "+decoded_output
            print(st)
            # print("\033[92mGenerated civil alternative-t5small:\033[0m", decoded_output)
            # print("\n")
          # else:
          #   # print("\033[91mGenerated civil alternative-t5small:\033[0m", "Couldn't generate")
          #   # print("\n")

      generated_civil_sentence = Bart_large(comment)
              # Add the comment to the list
      comments.append(generated_civil_sentence)
      # Create a pandas DataFrame from the collected comments
      data = {'message': comments, 'label': 0}
      df = pd.DataFrame(data)

      preds_flat_test = detection(df)

      if preds_flat_test[-1]==0:
          civil_options+=1
          st = "Option "+str(civil_options)+": "+generated_civil_sentence
          print(st)
          # print("\033[92mGenerated civil alternative-BART_large:\033[0m", generated_civil_sentence)
          # print("\n")
      else:
          # print("\033[91mGenerated civil alternative-BART_large:\033[0m", "Couldn't generate")
          # print("\n")

          generated_civil_sentence = Bart_base(comment)
              # Add the comment to the list
          comments.append(generated_civil_sentence)
          # Create a pandas DataFrame from the collected comments
          data = {'message': comments, 'label': 0}
          df = pd.DataFrame(data)
          preds_flat_test = detection(df)

          if preds_flat_test[-1]==0:
              civil_options+=1
              st = "Option "+str(civil_options)+": "+generated_civil_sentence
              print(st)
              # print("\033[92mGenerated civil alternative-BART_base:\033[0m", generated_civil_sentence)
              # print("\n")
          # else:
          #     print("\033[91mGenerated civil alternative-BART_base:\033[0m", "Couldn't generate")
          #     print("\n")



      #for nllb
      uncivil_sentence = comment
      inputs = tokenizer_nllb(uncivil_sentence, return_tensors="pt", max_length=1024, truncation=True).to(device)
      generated_ids = fine_tuned_nllb_model.generate(**inputs, max_new_tokens=1024)
      generated_civil_sentence = tokenizer_nllb.decode(generated_ids[0], skip_special_tokens=True)

              # Add the comment to the list
      comments.append(generated_civil_sentence)
      # Create a pandas DataFrame from the collected comments
      data = {'message': comments, 'label': 0}
      df = pd.DataFrame(data)

#         encoded_test_val = tokenizer.batch_encode_plus(
#         df.message.values,
#         add_special_tokens=True,
#         return_attention_mask=True,
#         pad_to_max_length=True,
#         max_length=512,
#         return_tensors='pt'
#     )

#         input_ids_test = encoded_test_val['input_ids']
#         attention_masks_test = encoded_test_val['attention_mask']
#         labels_test = torch.tensor(df.label.values)

#         dataset_test = TensorDataset(input_ids_test, attention_masks_test,labels_test)
#         batch_size = 8
#         dataloader_test = DataLoader(dataset_test,
#                                            sampler=SequentialSampler(dataset_test),
#                                            batch_size=batch_size)


#         pred_test = test_model(dataloader_test)
#         preds_flat_test = np.argmax(pred_test, axis=1).flatten()
      preds_flat_test = detection(df)

      if preds_flat_test[-1]==0:
          civil_options+=1
          st = "Option "+str(civil_options)+": "+generated_civil_sentence
          print(st)
          # print("\033[92mGenerated civil alternative-Nllb:\033[0m", generated_civil_sentence)
          # print("\n")
      # else:
      #     print("\033[91mGenerated civil alternative-Nllb:\033[0m", "Couldn't generate")
      #     print("\n")





      #for MarianMT
      uncivil_sentence = comment
      inputs = tokenizer_Marian(uncivil_sentence, return_tensors="pt", max_length=1024, trancation=True).to(device)
      generated_ids = fine_tuned_marian_model.generate(**inputs, max_new_tokens=1024)
      generated_civil_sentence = tokenizer_Marian.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
      subword_units = generated_civil_sentence.split('▁')
      # Remove empty strings resulting from the split
      subword_units = [unit for unit in subword_units if unit]
      # Join subword units into words
      cleaned_sentence = ' '.join(subword_units)

                      # Add the comment to the list
      comments.append(cleaned_sentence)
      # Create a pandas DataFrame from the collected comments
      data = {'message': comments, 'label': 0}
      df = pd.DataFrame(data)

#         encoded_test_val = tokenizer.batch_encode_plus(
#         df.message.values,
#         add_special_tokens=True,
#         return_attention_mask=True,
#         pad_to_max_length=True,
#         max_length=512,
#         return_tensors='pt'
#     )

#         input_ids_test = encoded_test_val['input_ids']
#         attention_masks_test = encoded_test_val['attention_mask']
#         labels_test = torch.tensor(df.label.values)

#         dataset_test = TensorDataset(input_ids_test, attention_masks_test,labels_test)
#         batch_size = 8
#         dataloader_test = DataLoader(dataset_test,
#                                            sampler=SequentialSampler(dataset_test),
#                                            batch_size=batch_size)


#         pred_test = test_model(dataloader_test)
#         preds_flat_test = np.argmax(pred_test, axis=1).flatten()
      preds_flat_test = detection(df)

      if preds_flat_test[-1]==0:
          civil_options+=1
          st = "Option "+str(civil_options)+": "+cleaned_sentence
          print(st)
          # print("\033[92mGenerated civil alternative-MarianMT:\033[0m", cleaned_sentence)
          # print("\n")
      # else:
      #     print("\033[91mGenerated civil alternative-MarianMT:\033[0m", "Couldn't generate")
      #     print("\n")

      if civil_options==0:
        print("Sorry, the model could not generate civil alternative for your provided comments.\n")

      else:
        civil_options=0

  print("\n\n")
# Finalize the experiment loop
print("Exiting the experiment loop.")


Enter a comment (or type 'exit' to stop): I feel this tuning is ugly... Could you show the concrete number of improvement? Have you do warmup before measure to fill pool?
This comment is professionally inappropriate!


Option 1: I find this tuning challenging. Could you provide a concrete number of improvements? Have you performed warmup before measuring to fill the pool?
Option 2: I feel this tuning is not aesthetically pleasing. Could you provide a concrete number of improvement? Have you conducted a warmup before measure to fill pool?
Option 3: I find this tuning approach to be quite messy. Could you provide some concrete details about the number of improvements made? Have you thoroughly warmed up before measuring to fill the pool?
Option 4: I'm feel this closings part is unappealing. Could you display the concrete number of improvement? Have you taken a hotup before measure to fill pool?





In [ ]:
import logging
import warnings
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
import torch
import pandas as pd
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer, AdamW
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import MarianMTModel, MarianTokenizer
import warnings
warnings.filterwarnings('ignore')


# Set the logging level for the transformers module
logging.getLogger("transformers").setLevel(logging.ERROR)
# Suppress specific FutureWarning messages
warnings.filterwarnings("ignore", category=FutureWarning, message="The `pad_to_max_length` argument is deprecated*")

# Initialize an empty list to store comments
comments = []

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)
label_dict = {0: 0, 1: 1}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                          num_labels=len(label_dict),
                                                          output_attentions=False,
                                                          output_hidden_states=False)

model.to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/MyModel_BERT_epoch_4.model', map_location=torch.device('cpu')))




input_csv_file = "/content/drive/MyDrive/bart_random2000.csv"  # Replace with your CSV file path
comments_df = pd.read_csv(input_csv_file, header=None, names=["comment"])

# Create an empty list to store comments and their associated outputs
score_outputs = []

count=0
for inp_sentence in comments_df["comment"]:
    comment = inp_sentence
    # Add the comment to the list
    comments.append(comment)

    # Create a pandas DataFrame from the collected comments
    data = {'message': comments, 'label': 0}
    df = pd.DataFrame(data)

    preds_flat_test = detection(df)


    if preds_flat_test[-1]==0:
        print("\033[92mThis comment is professionally sound!\033[0m")
        print("\n")
    else:
        print("\033[91mThis comment is professionally inappropriate!\033[0m")
        print("\n")
    count+=1
    print("Count: ",count)

print(preds_flat_test)

This comment is professionally sound!


Count:  1
This comment is professionally sound!


Count:  2
This comment is professionally inappropriate!


Count:  3
This comment is professionally sound!


Count:  4
This comment is professionally sound!


Count:  5
This comment is professionally sound!


Count:  6
This comment is professionally sound!


Count:  7
This comment is professionally sound!


Count:  8
This comment is professionally sound!


Count:  9
This comment is professionally sound!


Count:  10
This comment is professionally sound!


Count:  11
This comment is professionally sound!


Count:  12
This comment is professionally sound!


Count:  13
This comment is professionally sound!


Count:  14
This comment is professionally sound!


Count:  15
This comment is professionally inappropriate!


Count:  16
This comment is professionally sound!


Count:  17
This comment is professionally inappropriate!


Count:  18
This comment is professionally sound!


Count:  19
This comment is 